# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/2_feature_pipeline.ipynb)


## 🗒️ This notebook is divided into the following sections:
1. Parse Data
2. Feature Group Insertion

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import pandas as pd
from datetime import datetime
import time 
import requests

from functions import *

---

## <span style='color:#ff5f27'> 👮🏻‍♂️ API Keys

You will parse Air Quality data from [World Air Quality Index](https://aqicn.org/api/) site using your own credentials, so you have to [get an API-key](https://aqicn.org/data-platform/token/) from there.

Also, to be able parse weather data, you should get an API key from [VisualCrossing](https://www.visualcrossing.com/). You can use [this link](https://www.visualcrossing.com/weather-api).

### Don't forget to create an `.env` configuration file where all the necessary environment variables will be stored:
![](images/api_keys_env_file.png)

In [ ]:
date_today = datetime.now().strftime("%Y-%m-%d")

---

## <span style='color:#ff5f27'>  🧙🏼‍♂️ Parsing Data

In [ ]:
cities = ['Kyiv', 'Stockholm', 'Sundsvall', 'Malmo']

data_air_quality = [get_air_quality_data(city) for city in cities]

data_weather = [get_weather_data(city, date_today) for city in cities]

---

## <span style='color:#ff5f27'> 🧑🏻‍🏫 Dataset Preparation

#### <span style='color:#ff5f27'> 👩🏻‍🔬 Air Quality Data

In [ ]:
df_air_quality = get_air_quality_df(data_air_quality)

df_air_quality

#### <span style='color:#ff5f27'> 🌦 Weather Data

In [ ]:
df_weather = get_weather_df(data_weather)

df_weather.head()

---

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

---

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
air_quality_fg.insert(df_air_quality)

In [ ]:
weather_fg.insert(df_weather)

---